# Reinforcement Learning for Navigation

In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed


from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper

In [2]:
ENV_PATH = r"C:\Users\user\Desktop\RLNav\RLAgency\Builds\P0\BasicSingle\Env.exe"

unity_env = UnityEnvironment(ENV_PATH, base_port=6008)
env = UnityToGymWrapper(unity_env)

2021-04-08 13:42:10 INFO [environment.py:111] Connected to Unity environment with package version 1.8.1-preview and communication version 1.4.0
2021-04-08 13:42:11 INFO [environment.py:279] Connected new brain:
Basic?team=0
2021-04-08 13:42:11 WARNING [__init__.py:91] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.
C:\Users\user\Anaconda3\envs\rlnav\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [22]:
def make_env(rank, seed=0):
    def _init():
        unity_env = UnityEnvironment(ENV_PATH, base_port=6030 + rank)
        env = UnityToGymWrapper(unity_env)
        env.seed(seed + rank)
        return env
    set_random_seed(seed)
    return _init


In [23]:
num_cpu = 4  # Number of processes to use
env = DummyVecEnv([make_env(i) for i in range(num_cpu)])

model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=25000)

2021-04-08 13:50:00 INFO [environment.py:111] Connected to Unity environment with package version 1.8.1-preview and communication version 1.4.0
2021-04-08 13:50:01 INFO [environment.py:279] Connected new brain:
Basic?team=0
2021-04-08 13:50:01 WARNING [__init__.py:91] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.
2021-04-08 13:50:01 WARNING [__init__.py:283] Could not seed environment Basic?team=0
2021-04-08 13:50:05 INFO [environment.py:111] Connected to Unity environment with package version 1.8.1-preview and communication version 1.4.0
2021-04-08 13:50:05 INFO [environment.py:279] Connected new brain:
Basic?team=0
2021-04-08 13:50:05 WARNING [__init__.py:91] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only t

Using cpu device
----------------------------------------
| time/                   |            |
|    fps                  | 119        |
|    iterations           | 1          |
|    time_elapsed         | 68         |
|    total_timesteps      | 8192       |
| train/                  |            |
|    approx_kl            | 0.01579943 |
|    clip_fraction        | 0.162      |
|    clip_range           | 0.2        |
|    entropy_loss         | -4.25      |
|    explained_variance   | 0.355      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0627    |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0238    |
|    std                  | 0.992      |
|    value_loss           | 0.00072    |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 111         |
|    iterations           | 2           |
|    time_elapsed         | 146     

ValueError: not enough values to unpack (expected 4, got 1)

In [25]:
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)

[-0.00097656 -0.08737247  0.01234111 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.05415308]
[ 0.04993232 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656  0.0639666   0.04619274 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.09498   ]
[-0.03255756 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656  0.06518778 -0.03290108 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.07869169]
[ 0.0247014  -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656  0.00445315  0.00403461 -0.00097656]


[-9.765625e-04 -9.765625e-04 -9.765625e-04  1.000000e+00]
[-0.05851147 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656  0.08535321  0.07443845 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.03621101]
[-0.00371423 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.06361452 -0.11562473 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.0297955 ]
[-0.09237789 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656  0.04504027  0.04646921 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.06

[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.05297124 -0.00097656]
[-0.00097656  0.09636669 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.00931953]
[ 0.03470439 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.03023119 -0.04796639 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.06586337]
[-0.04940134 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.08176449  0.07867146 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.06249455]
[ 0.05768771 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]


[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656  0.07631177  0.04396628 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.0860759 ]
[ 0.04487626 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656  0.00121152 -0.00097656]
[-0.00097656  0.01793811 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.01891854]
[-0.09730318 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[ 0.         -0.00097656  0.         -0.00097656]
[-0.00097656 -0.00097656 -0.0641548  -0.00097656]
[-0.00097656  0.08671536 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.05705234]
[ 0.07631737 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]


[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656  0.04796401  0.03699563 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.09382175]
[-0.08091303 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.02255911  0.05467021 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.        ]
[ 0.06930495 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656  0.04467156  0.00180227 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.09482151]
[ 0.01324444 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]


[-0.00097656 -0.05358896 -0.0908227  -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[ 0.02523089 -0.00097656 -0.00097656  0.02500719]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656  0.06142577 -0.00097656]
[-0.00097656 -0.04066479 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.06584655]
[ 0.04583188 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656  0.02595505 -0.02742802 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656  0.02373284]
[-0.00993182 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.00097656 -0.00097656 -0.00097656]
[-0.00097656 -0.09568718 -0.01449016 -0.00097656]
